## C-M ratio implementation ##


In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.io.img_tiles import OSM
from cartopy.io.img_tiles import GoogleTiles as moa

import glob

import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr
import pyproj
import osr
import datetime
import subprocess
import shutil
import numpy as np
from ipywidgets import interact

# import our own small lib
import nsidc

We assume there is a dataset available to work with (downloaded with NSIDC_Measures.ipynb). If you have retrieved a complete dataset over multiple years, please continue with the following part. We will:
- extract a time series of brightness temperatures over a river section
- extract time series in the surroundings of river section.
- apply the C/M ratio method (see Brakenridge et al., 2007, http://onlinelibrary.wiley.com/doi/10.1029/2006WR005238/full

We start by defining gauging points of interest, and transform these into the projection system of the Measures grids.

In [ ]:

proj4str = '+proj=cea +lat_0=0 +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m'
points_interest = [(-6.565704345703125, 13.366907166629518),
                   (-2.8667449951171875, 16.678293098288513),
]

# we define a projection object for lat-lon WGS84 (EPSG code 4326)
proj_out = pyproj.Proj(init='epsg:4326')
# we define a projection object for the projection used in the downloaded grids.
proj_in = pyproj.Proj(proj4str)

# here we convert the coordinates in lat-lon into the coordinate system of the downloaded grids.
points_xy = nsidc.proj_coords(points_interest, proj_out, proj_in)


In [ ]:
# first have a look at the points
osm_tiles = moa()
x, y = zip(*points_interest)

plt.figure(figsize=(8, 8))

# Use the tile's projection for the underlying map.
ax = plt.axes(projection=osm_tiles.crs)

# Specify a region of interest, in this case, Cardiff.
# ax.set_extent([1, 3, 6, 9],
#               ccrs.PlateCarree())

# Add the tiles at zoom level 12.
ax.add_image(osm_tiles, 10)
ax.plot(x, y, color='r', marker='.', markersize=20, linewidth=0., transform=ccrs.PlateCarree())
# ax.coastlines('10m')

plt.show()

In [ ]:
fns = glob.glob(r'd:\git\satellite-cookbook\NSIDC-AMSRE\netcdf\*.nc')
fns.sort()
fns

In [ ]:
ds = xr.open_mfdataset(fns[0:5])
points_x, points_y = zip(*points_xy)

ts = ds.sel_points(x=list(points_x), y=list(points_y), method='nearest')
# From variable 'TB' let's plot time series sequentially
ax = plt.subplot(111)
for n in range(len(ts['TB'])):
    ts['TB'][n].plot(ax=ax, marker='.', linewidth=0., label='point {:d}'.format(n + 1))
# ax.legend()
plt.title('Brightness temperatures at locations')
# list(points_x)
# ds.y

Prepare for the selected points of interest a time series. We will save these in a list of time series 's'. After that we interactively plot the data. You can select which point you are looking at, which date, which time window around that date. Alongside the plotted date, you will see (right-hand side) a plot of the brightness temperatures over the point of interest. To-Do: make the spatial window around the point displayed also configurable.

In [ ]:

s = [nsidc.c_m_ratio(ds['TB'], x, y)[2] for x, y in zip(points_x, points_y)]
#     s = [c_m_ratio(ds_win['TB'], x, y)[2] for ds_win, x, y in zip(window_ts, points_x, points_y)]

def plot_func(idx, point, t_window, vmin, vmax):
    t = s[point].time.values[int(idx)]
    f = plt.figure(figsize=(12,6))
    ax1 = f.add_subplot(121)
    s[point].sel(time=slice(t- np.timedelta64(t_window, 'D'), t + np.timedelta64(t_window, 'D'))).plot(marker='.', linewidth=0.)
    plt.plot([t, t], ax1.get_ylim(), 'r')
    ax2 = f.add_subplot(122, projection=ccrs.PlateCarree())

    ax2.add_feature(cfeature.LAND)
    ax2.add_feature(cfeature.OCEAN)
    # get a mesh of the coordinates in grid's projection

    # now drape the data on the map
    p = ax2.pcolormesh(loni, lati, ds.sel(time=t)['TB'], transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
    # plot the current location in lat-lon
    p2 = plt.plot(*nsidc.proj_coord((s[point].x, s[point].y), proj_in, proj_out), marker='x', color='r')
    # also plot some points of interest 
    plt.colorbar(p, label='K')
    gl = ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=1, color='gray', alpha=0.5, linestyle=':')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER

xi, yi = np.meshgrid(ds.x, ds.y)

loni, lati = pyproj.transform(proj_in, proj_out, xi, yi)

# below we interactively plot!
interact(plot_func,
         idx=(0., len(s[0]), 1.),
         point=range(len(s)),
         t_window=(10, 730, 1),
         vmin=(230, 290, 1),
         vmax=(250, 310, 1),
         continuous_update=False,
        )
